In [1]:
import pandas as pd

In [2]:
def load_data(path="../data/csv/all.csv"):
    return pd.read_csv(path, index_col=0)

In [3]:
def vectorize_data(data_frame):
    # TODO - implement full vectorization of df
    # inputs -> number of inputs
    # outputs -> number of outputs
    # etc
    pass

In [4]:
df = load_data()

In [5]:
df.head()

,jobId,read_sum,write_sum,readSyscalls_sum,writeSyscalls_sum,readReal_sum,writeReal_sum,writeCancelled_sum,rxBytes_sum,rxPackets_sum,...,mem.used,mem.active,mem.available,mem.buffers,mem.cached,mem.slab,mem.buffcache,mem.swaptotal,mem.swapused,mem.swapfree
0,-8Ip2oamm-1-10,1.640063e+07,1865.0,20775.0,37.0,3.399680e+05,69632.0,0.0,0.0,0.0,...,3326259200,941318144,7053393920,2768896,2302001152,590626816,2895396864,0,0,0
1,-8Ip2oamm-1-11,3.943547e+09,99029.0,863138.0,1382.0,2.988442e+09,4825088.0,0.0,506728.0,1192.0,...,3656019968,1019281408,7034150912,2768896,2545008640,594448384,3142225920,0,0,0
2,-8Ip2oamm-1-12,2.260023e+09,63886.0,561366.0,906.0,1.886495e+09,2158592.0,0.0,341954.0,803.0,...,3795075072,995864576,7057567744,2768896,2772025344,480731136,3255525376,0,0,0
3,-8Ip2oamm-1-13,2.284814e+09,147790.0,582911.0,29318.0,4.472832e+06,3805184.0,0.0,354552.0,832.0,...,4242853888,1047011328,6947700736,2768896,3209531392,467460096,3679760384,0,0,0
4,-8Ip2oamm-1-14,2.754452e+09,70983.0,628488.0,986.0,2.200519e+09,3858432.0,0.0,380100.0,894.0,...,3600007168,966131712,7087300608,2768896,2544816128,590618624,3138203648,0,0,0


In [6]:
df.columns

Index(['jobId', 'read_sum', 'write_sum', 'readSyscalls_sum',
       'writeSyscalls_sum', 'readReal_sum', 'writeReal_sum',
       'writeCancelled_sum', 'rxBytes_sum', 'rxPackets_sum', 'rxErrors_sum',
       'rxDrop_sum', 'rxFifo_sum', 'rxFrame_sum', 'rxCompressed_sum',
       'rxMulticast_sum', 'txBytes_sum', 'txPackets_sum', 'txErrors_sum',
       'txDrop_sum', 'txFifo_sum', 'txColls_sum', 'txCarrier_sum',
       'txCompressed_sum', 'cpu_mean', 'cpu_max', 'memory_mean', 'memory_max',
       'ctime_mean', 'ctime_max', 'ctime_sum', 'workflowName', 'size',
       'executable', 'args', 'inputs', 'outputs', 'name', 'command',
       'execTimeMs', 'cpu.manufacturer', 'cpu.brand', 'cpu.vendor',
       'cpu.family', 'cpu.model', 'cpu.stepping', 'cpu.speed', 'cpu.cores',
       'cpu.physicalCores', 'cpu.processors', 'cpu.cache.l1d', 'cpu.cache.l1i',
       'cpu.cache.l2', 'cpu.cache.l3', 'mem.total', 'mem.free', 'mem.used',
       'mem.active', 'mem.available', 'mem.buffers', 'mem.cached', 'mem

In [65]:
def normalize_column(column):
    minvalue, maxvalue = column.min(), column.max()
    return (column - minvalue) / (maxvalue - minvalue)

def vectorize_textual(column):
    return pd.get_dummies(column)

df.select_dtypes(include='object')
df.ctime_max.mean(), df.execTimeMs.mean()

(11051.542998600797, 12287.035978192425)

## I co dalej
Było
- workflow
- size
- inputs
- outputs
- name
- cpu manufacturer
- cpu brand
- cpu speed
- cpu cores
- cpu physical cores
- mem_total
- mem_active

Wynikowo:
- memory_max <- czy memory intensive
- execTimeMs <- czy cpu intensive
- read_sum <- czy read intensive
- write_sum <- czy write intensive

Czyli - wektoryzujemy wszystko oprócz memory_max, execTimeMs, read_sum i write_sum

In [8]:
df.select_dtypes(exclude='object').dtypes

read_sum              float64
write_sum             float64
readSyscalls_sum      float64
writeSyscalls_sum     float64
readReal_sum          float64
writeReal_sum         float64
writeCancelled_sum    float64
rxBytes_sum           float64
rxPackets_sum         float64
rxErrors_sum          float64
rxDrop_sum            float64
rxFifo_sum            float64
rxFrame_sum           float64
rxCompressed_sum      float64
rxMulticast_sum       float64
txBytes_sum           float64
txPackets_sum         float64
txErrors_sum          float64
txDrop_sum            float64
txFifo_sum            float64
txColls_sum           float64
txCarrier_sum         float64
txCompressed_sum      float64
cpu_mean              float64
cpu_max               float64
memory_mean           float64
memory_max            float64
ctime_mean            float64
ctime_max             float64
ctime_sum             float64
size                  float64
execTimeMs            float64
cpu.family            float64
cpu.model 

In [9]:
test_df = df[:50]
vectorize_textual(df['cpu.manufacturer'])
vectorize_numerical(df['cpu.cache.l3'])

0         1.387163e-07
1         3.294512e-07
2         3.294512e-07
3         1.387163e-07
4         3.294512e-07
              ...     
185815    3.814698e-07
185816    3.814698e-07
185817    3.814698e-07
185818    3.814698e-07
185819    3.814698e-07
Name: cpu.cache.l3, Length: 185820, dtype: float64

In [131]:
def results_frame(dataframe):
    return dataframe.filter(['execTimeMs', 'read_sum', 'write_sum', 'memory_max'])

def features_frame(dataframe):
    filtered = dataframe.filter(['workflowName', 'size', 'executable', 'args', 'inputs', 'outputs', 'name', 'cpu.manufacturer', 'cpu.brand', 'cpu.vendor',
                               'cpu.family', 'cpu.model', 'cpu.stepping', 'cpu.speed', 'cpu.cores', 'cpu.physicalCores', 'cpu.processors', 'cpu.cache.l1d', 
                               'cpu.cache.l1i', 'cpu.cache.l2', 'cpu.cache.l3', 'mem.total', 'mem.free', 'mem.used', 'mem.active', 'mem.available', 'mem.buffers', 
                               'mem.cached', 'mem.slab', 'mem.buffcache', 'mem.swaptotal', 'mem.swapused', 'mem.swapfree'])
    dataframe = drop_null_columns(filtered)
    dataframe = drop_boring_columns(dataframe)    
    
    vectorize_list_columns(dataframe, ['args', 'inputs', 'outputs'])
    
    numerical_columns = dataframe.select_dtypes(include='number').columns  
    vectorize_numerical_columns(dataframe, numerical_columns)
    
    textual_columns = dataframe.select_dtypes(include="object").columns
    dataframe = vectorize_textual_columns(dataframe, textual_columns)
    return dataframe
    
def drop_null_columns(dataframe):
    null_columns = get_columns_with_nulls(dataframe)
    return dataframe.drop(null_columns, axis=1)

def drop_boring_columns(dataframe):
    boring_columns = get_non_variable_columns(dataframe)
    return dataframe.drop(boring_columns, axis=1)
    
def vectorize_list_columns(dataframe, columns):
    for column in columns:
        dataframe[column] = vectorize_list(dataframe[column])
        
def vectorize_numerical_columns(dataframe, columns):
    for column in columns:
        dataframe[column] = normalize_column(dataframe[column])
        
def vectorize_textual_columns(dataframe, columns):
    encoded = [pd.get_dummies(dataframe[column]) for column in columns]    
    return pd.concat([dataframe.drop(columns, axis=1), *encoded], axis=1)
    
def vectorize_list(series):
    def vectorize(list_string):
        return len(eval(list_string))
    return series.apply(vectorize)
    
def get_columns_with_nulls(dataframe):
    columns = list(dataframe.columns)
    return [column for column in columns if dataframe[column].isnull().values.any()]

def get_non_variable_columns(dataframe):
    columns = list(dataframe.columns)
    return [column for column in columns if dataframe[column].unique().size == 1]

In [133]:
test_df = df[:1000]
dummies = pd.get_dummies(test_df['name'])
# test_df.join(dummies)
dummies.columns

features_frame(df)

,size,args,inputs,outputs,cpu.speed,cpu.cores,cpu.physicalCores,mem.total,mem.free,mem.used,...,seq_dict,sort_sam,AMD,Intel®,EPYC 7B12,Xeon®,Xeon® Platinum 8124M,Xeon® Platinum 8175M,Xeon® Platinum 8259CL,Xeon® Platinum 8275CL
0,0.085613,0.163636,0.009675,0.25,0.909091,0.333333,0.142857,0.090152,0.070839,0.103684,...,0,0,0,1,0,0,1,0,0,0
1,0.085613,0.272727,0.008795,0.25,0.909091,0.333333,0.142857,0.091049,0.066667,0.119717,...,0,0,0,1,0,0,0,0,0,1
2,0.085613,0.272727,0.008795,0.25,0.909091,0.333333,0.142857,0.091049,0.064527,0.126478,...,0,0,0,1,0,0,0,0,0,1
3,0.085613,0.272727,0.008795,0.25,0.909091,0.333333,0.142857,0.090152,0.056732,0.148248,...,0,0,0,1,0,0,1,0,0,0
4,0.085613,0.272727,0.008795,0.25,0.909091,0.333333,0.142857,0.091049,0.067529,0.116994,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185815,0.125369,0.200000,0.001759,0.25,0.000000,0.333333,0.142857,0.483365,0.464132,0.112833,...,0,0,0,1,0,1,0,0,0,0
185816,0.125369,0.200000,0.007916,0.25,0.000000,0.333333,0.142857,0.483365,0.421357,0.247954,...,0,0,0,1,0,1,0,0,0,0
185817,0.125369,0.200000,0.007916,0.00,0.000000,0.333333,0.142857,0.483365,0.442584,0.180902,...,0,0,0,1,0,1,0,0,0,0
185818,0.125369,0.200000,0.007916,0.25,0.000000,0.333333,0.142857,0.483365,0.463316,0.115412,...,0,0,0,1,0,1,0,0,0,0


# TODO - perhaps use multiindex, but might not be of use actually